In [19]:
from gensim.models.poincare import PoincareModel, PoincareKeyedVectors, PoincareRelations
from  gensim.viz.poincare import poincare_2d_visualization
from gensim.test.utils import datapath


###  Steps :
1. generate word and adjective syncset closure from wordnet 
2. use poincare embedding from gensim to create embedding 
3. convert poincare embedding into wordwvec format
4. use word2vec format of poincare embedding to train model 



#### create closure 

In [20]:
import re
import pandas
from nltk.corpus import wordnet as wn
from tqdm import tqdm
try:
    wn.all_synsets
except LookupError as e:
    import nltk
    nltk.download('wordnet')

# make sure each edge is included only once
edges = set()
poss = ['a', 'n']
for p in poss :
    for synset in tqdm(wn.all_synsets(pos=p)):
        # write the transitive closure of all hypernyms of a synset to file
        for hyper in synset.closure(lambda s: s.hypernyms()):
            edges.add((synset.name(), hyper.name()))

        # also write transitive closure for all instances of a synset
        for instance in synset.instance_hyponyms():
            for hyper in instance.closure(lambda s: s.instance_hypernyms()):
                edges.add((instance.name(), hyper.name()))
                for h in hyper.closure(lambda s: s.hypernyms()):
                    edges.add((instance.name(), h.name()))

nouns_and_adjectives  = pandas.DataFrame(list(edges), columns=['id1', 'id2'])
#nouns_and_adjectives['weight'] = 1
nouns_and_adjectives.to_csv('nouns_and_adjectives_closure.csv', index=False)


18156it [00:00, 41382.54it/s]
82115it [00:05, 15049.06it/s]


### create poincare embedding

qwwq

In [21]:
wordnet_mamal_file_path = 'nouns_and_adjectives_closure.csv'
relations = PoincareRelations(wordnet_mamal_file_path, delimiter=',')

In [22]:
model = PoincareModel(train_data=relations, size= 2 , burn_in=0)
model.train(epochs = 2 , print_every = 500)

In [23]:
model.kv.distance('mammal.n.01', 'placental.n.01')

0.14510816413473474

In [24]:
#poincare_2d_visualization(model , relations , 'aaa', show_node_labels= model.kv.vocab.keys())

In [25]:
model.kv.difference_in_hierarchy('mammal.n.01', 'dog.n.01')

-0.07578675645177146

In [26]:
model.kv.most_similar('kangaroo.n.01', topn=2)

[('allen.n.01', 0.0012030596733936267), ('potato.n.01', 0.0036251920706466025)]

In [27]:
model.kv.norm('mammal.n.01')

0.31622202906152214

###  create word2wec form embedding from poincare embedding

In [34]:
model.kv.save_word2vec_format('word2_wec_poinc', fvocab=None, binary=False, total_vec=None)

In [29]:
#### create model poincare embedding 

In [30]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [31]:
with open('word2_wec_poinc', 'r') as f:
    emb= f.readlines()
    

In [32]:
#E = np.loadtxt('word2_wec_poinc.tsv', delimiter='\t')
embedding_layer = Embedding(100, 10, input_length=100, weights = emb)
embedding_layer.trainable = True

In [33]:
emb

['82117 2\n',
 'entity.n.01 0.15088413981304913 -0.06334821782735708\n',
 'physical_entity.n.01 -0.15911052935053496 0.10425909984951366\n',
 'abstraction.n.06 0.0929000250726278 -0.09924813547481226\n',
 'object.n.01 -0.05827593265622807 0.11848379384715858\n',
 'whole.n.02 -0.002793656784416539 0.2016839370637566\n',
 'living_thing.n.01 -0.011661136982592489 0.23299639480925122\n',
 'organism.n.01 0.016997431923719622 0.15935354648185138\n',
 'psychological_feature.n.01 0.05156238386575099 -0.1815442309897632\n',
 'causal_agent.n.01 -0.06160358446167021 0.13221582588611458\n',
 'artifact.n.01 -0.006605711267705731 0.062762084022507\n',
 'person.n.01 0.1102568784695472 0.05175232588189928\n',
 'group.n.01 0.036634512156849104 -0.14429788811969604\n',
 'event.n.01 0.08112750275945167 -0.2516279438548307\n',
 'attribute.n.02 0.011598103507409821 -0.08145886567258653\n',
 'act.n.02 0.17142877265927595 -0.2303645323395997\n',
 'matter.n.03 -0.15064630010524424 0.17846873263889854\n',
 'in